# Sentiment Based Product recommendation system

## 1. Importing necessary libraries

In [102]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from itertools import chain
from fractions import Fraction

import os
from nltk.corpus import stopwords
import string
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import operator
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import time
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.cluster import AgglomerativeClustering


In [101]:
!pip install chart_studio

     |████████████████████████████████| 64 kB 1.9 MB/s 


In [78]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [80]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

## 2.1. Data Loading

In [4]:
# Connecting to local google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/product-recommendation-system/dataset/sample30.csv')

In [50]:
df

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-23T00:00:00.000Z,False,True,5,I got this conditioner with Influenster to try...,Softness!!,NaN,NaN,laurasnchz,Positive
29996,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-27T00:00:00.000Z,False,True,5,"I love it , I received this for review purpose...",I love it,NaN,NaN,scarlepadilla,Positive
29997,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-21T00:00:00.000Z,False,True,5,First of all I love the smell of this product....,Hair is so smooth after use,NaN,NaN,liviasuexo,Positive
29998,AVpfW8y_LJeJML437ySW,L'oreal Paris,"Beauty,Hair Care,Shampoo & Conditioner,Holiday...",L'oreal Paris,L'or233al Paris Elvive Extraordinary Clay Reba...,2017-01-11T00:00:00.000Z,False,True,5,I received this through Influenster and will n...,Perfect for my oily hair!,NaN,NaN,ktreed95,Positive


## 2.2. Data cleaning, to handle missing values and to remove unnecessary columns

In [51]:
df.info

<bound method DataFrame.info of                          id            brand  ... reviews_username user_sentiment
0      AV13O1A8GV-KLJ3akUyj  Universal Music  ...           joshua       Positive
1      AV14LG0R-jtxr-f38QfS         Lundberg  ...        dorothy w       Positive
2      AV14LG0R-jtxr-f38QfS         Lundberg  ...        dorothy w       Positive
3      AV16khLE-jtxr-f38VFn              K-Y  ...          rebecca       Negative
4      AV16khLE-jtxr-f38VFn              K-Y  ...        walker557       Negative
...                     ...              ...  ...              ...            ...
29995  AVpfW8y_LJeJML437ySW    L'oreal Paris  ...       laurasnchz       Positive
29996  AVpfW8y_LJeJML437ySW    L'oreal Paris  ...    scarlepadilla       Positive
29997  AVpfW8y_LJeJML437ySW    L'oreal Paris  ...       liviasuexo       Positive
29998  AVpfW8y_LJeJML437ySW    L'oreal Paris  ...         ktreed95       Positive
29999  AVpfW8y_LJeJML437ySW    L'oreal Paris  ...        kcoopxoxo

In [52]:
df['reviews_rating'].value_counts()

5    20831
4     6020
1     1384
3     1345
2      420
Name: reviews_rating, dtype: int64

In [53]:
df.isnull().sum()

id                          0
brand                       0
categories                  0
manufacturer              141
name                        0
reviews_date               46
reviews_didPurchase     14068
reviews_doRecommend      2570
reviews_rating              0
reviews_text                0
reviews_title             190
reviews_userCity        28071
reviews_userProvince    29830
reviews_username           63
user_sentiment              1
dtype: int64

In [54]:
df.drop_duplicates(subset=['reviews_username','reviews_date'],inplace=True)

In [55]:
# Dropping User city & user province
df.drop(['reviews_userCity','reviews_userProvince','id'], axis=1, inplace=True)

In [56]:
df.shape

(29118, 12)

We have fairly compact columns & compact data now.

In [57]:
# Getting list of columns
df_l = list(df.columns)
df_l

['brand',
 'categories',
 'manufacturer',
 'name',
 'reviews_date',
 'reviews_didPurchase',
 'reviews_doRecommend',
 'reviews_rating',
 'reviews_text',
 'reviews_title',
 'reviews_username',
 'user_sentiment']

In [58]:
df.isnull().sum()

brand                      0
categories                 0
manufacturer             141
name                       0
reviews_date              41
reviews_didPurchase    13930
reviews_doRecommend     2539
reviews_rating             0
reviews_text               0
reviews_title            186
reviews_username          57
user_sentiment             1
dtype: int64

In [59]:
df = df.dropna(axis=0, subset=['user_sentiment'])

In [72]:
df_t = pd.DataFrame(df, columns=['reviews_rating','reviews_text','reviews_title','reviews_username','user_sentiment'])

#### Seems data is fairly clean for our purpose

## 3. Text preprocessing

In [64]:
# Punctuation removal
def remove_punct(df):
    count = 0
    for s in df:
        cleanr = re.compile('<.*?>')
        s = re.sub(r'\d+', '', s)
        s = re.sub(cleanr, '', s)
        s = re.sub("'", '', s)
        s = re.sub(r'\W+', ' ', s)
        s = s.replace('_', '')
        df.loc[count] = s
        count+=1
    return(df)

In [65]:
# Stop words removal
def remove_stw(df):
    # Downloading english stop words data
    stop_words = set(stopwords.words('english'))

    # Removing Stop words 
    count = 0
    for sentence in df:
        sentence = [word for word in sentence.lower().split() if word not in stop_words]
        sentence = ' '.join(sentence)
        df.loc[count] = sentence
        count+=1
    return(df)

In [66]:
# Lemmatization
def lemma(df):

    lmtzr = WordNetLemmatizer()

    count = 0
    stemmed = []
    for sentence in df:    
        word_tokens = word_tokenize(sentence)
        for word in word_tokens:
            stemmed.append(lmtzr.lemmatize(word))
        sentence = ' '.join(stemmed)
        df.iloc[count] = sentence
        count+=1
        stemmed = []
    return(df)

In [67]:
# Stemmatization
def stemma(df):

    stemmer = SnowballStemmer("english") #SnowballStemmer("english", ignore_stopwords=True)

    count = 0
    stemmed = []
    for sentence in df:
        word_tokens = word_tokenize(sentence)
        for word in word_tokens:
            stemmed.append(stemmer.stem(word))
        sentence = ' '.join(stemmed)
        df.iloc[count] = sentence
        count+=1
        stemmed = []
    return(df)

In [81]:
df_t['reviews_text']= remove_punct(df_t['reviews_text'])
df_t['reviews_text']= remove_stw(df_t['reviews_text'])
df_t['reviews_text']= lemma(df_t['reviews_text'])
df_t['reviews_text']= stemma(df_t['reviews_text'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


## 4. Feature extraction or feature creation

In [92]:
def get_feature(df, number):
    
    feature_list = []
    # create an instance for tree feature selection
    tree_clf = ExtraTreesClassifier()

    # first create arrays holding input and output data

    # Vectorizing Train set
    cv = CountVectorizer(analyzer='word')
    x_train = cv.fit_transform(df['reviews_text'])

    # Creating an object for Label Encoder and fitting on target strings
    le = LabelEncoder()
    y = le.fit_transform(df['label'])

    # fit the model
    tree_clf.fit(x_train, y)
    
    # Preparing variables
    importances = tree_clf.feature_importances_
    feature_names = cv.get_feature_names()
    feature_imp_dict = dict(zip(feature_names, importances))
    sorted_features = sorted(feature_imp_dict.items(), key=operator.itemgetter(1), reverse=True)
    indices = np.argsort(importances)[::-1]

    # Create the feature list
    for f in range(number):
        feature_list.append(sorted_features[f][0])
    
    return(feature_list)

### Getting the features list and applying the udf on our data

In [93]:
def print_feature(df):
    
    # create an instance for tree feature selection
    tree_clf = ExtraTreesClassifier()

    # first create arrays holding input and output data

    # Vectorizing Train set
    cv = CountVectorizer(analyzer='word')
    x_train = cv.fit_transform(df['reviews_text'])

    # Creating an object for Label Encoder and fitting on target strings
    le = LabelEncoder()
    y = le.fit_transform(df['label'])

    # fit the model
    tree_clf.fit(x_train, y)

    # Preparing variables
    importances = tree_clf.feature_importances_
    feature_names = cv.get_feature_names()
    feature_imp_dict = dict(zip(feature_names, importances))
    sorted_features = sorted(feature_imp_dict.items(), key=operator.itemgetter(1), reverse=True)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")
    for f in range(20):
        print("feature %d : %s (%f)" % (indices[f], sorted_features[f][0], sorted_features[f][1]))

    # Plot the feature importances of the forest
    plt.figure(figsize = (20,20))
    plt.title("Feature importances")
    plt.bar(range(100), importances[indices[:100]],
           color="g", align="center")
    plt.xticks(range(100), sorted_features[:100], rotation=90)
    plt.xlim([-1, 100])
    plt.show()

    return()

In [94]:
# Getting best feats
def get_bestrf(X, y):
    parameters = [
        {
            "n_estimators":[5, 10, 20, 50, 100],
            "criterion":['gini', 'entropy']
        }
    ]

    best_clf = GridSearchCV(clf, parameters, scoring="accuracy", verbose=5, n_jobs=4)

    best_clf.fit(X, y)
    
    return(best_clf.best_estimator_.n_estimators, best_clf.best_estimator_.criterion)


In [103]:
from sklearn.ensemble import ExtraTreesClassifier

print_feature(df_t)

KeyError: ignored

## 5. Training of text classification models for sentiment analysis using the ML techniques                  such as logistic regression, random forest, XGBoost and Naive Bayes algorithm

       

In [104]:
df_t

,reviews_rating,reviews_text,reviews_title,reviews_username,user_sentiment
0,5,love album good hip hop side current pop sound...,Just Awesome,joshua,Positive
1,5,good flavor review collect part promot,Good,dorothy w,Positive
3,1,bought tri wife enjoy feel anyth enhanc produc...,Disappointed,rebecca,Negative
4,1,bought tri spice thing felt noth disappoint ho...,Irritation,walker557,Negative
5,1,first time use k didnt anyth opinion felt like...,Not worth it,samantha,Negative
...,...,...,...,...,...
29995,5,i got this condition with influenst to tri it ...,Softness!!,laurasnchz,Positive
29996,5,"i love it , i receiv this for review purpos fr...",I love it,scarlepadilla,Positive
29997,5,first of all i love the smell of this product ...,Hair is so smooth after use,liviasuexo,Positive
29998,5,i receiv this through influenst and will never...,Perfect for my oily hair!,ktreed95,Positive


In [119]:
X = df['reviews_text']
y = df['reviews_rating']

In [120]:
c = CountVectorizer(stop_words = 'english')
def text_fit(X, y, model,clf_model,coef_show=1):
    
    X_c = model.fit_transform(X)
    #print(X_c)
    print('# features: {}'.format(X_c.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=42)
    print('# train records: {}'.format(X_train.shape[0]))
    print('# test records: {}'.format(X_test.shape[0]))
    
    
    
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print ('Model Accuracy: {}'.format(acc))
    
    if coef_show == 1: 
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('-Top 20 positive-')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('-Top 20 negative-')        
        print(coeff_df.tail(20).to_string(index=False))
    
text_fit(X, y, c, LogisticRegression(solver='saga', max_iter=10000))    
#text_fit(X, y, c, LogisticRegression())

# features: 17231
# train records: 21837
# test records: 7280
Model Accuracy: 0.7365384615384616

-Top 20 positive-
           Word  Coefficient
         boring     1.640664
          worst     1.524867
       terrible     1.519445
        formula     1.340974
          waste     1.309417
       contains     1.273842
       horrible     1.267747
          money     1.261101
            sad     1.225532
         offers     1.211928
   conditionaer     1.164868
         broken     1.145505
          awful     1.143227
          total     1.112875
 disappointment     1.104619
     absolutely     1.095899
          batch     1.094269
        plugged     1.089547
           poor     1.083607
          worse     1.082186

-Top 20 negative-
      Word  Coefficient
      save    -0.777795
     helps    -0.783193
 boyfriend    -0.783844
     heads    -0.790625
    easily    -0.810725
      high    -0.827077
      nice    -0.832527
   pantene    -0.845553
  favorite    -0.855725
 excellent    -0

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [121]:
text_fit(X, y, c, DummyClassifier(),0)


# features: 17231
# train records: 21837
# test records: 7280
Model Accuracy: 0.6879120879120879


In [122]:
tfidf = TfidfVectorizer(stop_words = 'english')
text_fit(X, y, tfidf, LogisticRegression(solver='saga', max_iter=10000))

# features: 17231
# train records: 21837
# test records: 7280
Model Accuracy: 0.7355769230769231

-Top 20 positive-
         Word  Coefficient
      formula     4.134702
          gel     3.131771
          new     2.657767
     terrible     2.650936
     horrible     2.560716
        money     2.354074
      product     2.209837
        worst     2.142141
 disappointed     2.037237
        waste     1.951239
          old     1.901524
       broken     1.881790
        awful     1.825635
        worse     1.800462
        total     1.711177
       change     1.699604
          sad     1.677725
       boring     1.660573
         poor     1.625423
        scent     1.604155

-Top 20 negative-
      Word  Coefficient
    better    -1.093281
     price    -1.096747
       use    -1.124064
    little    -1.163438
     clean    -1.219130
    action    -1.237997
     color    -1.279567
     works    -1.304193
      soft    -1.527195
      best    -1.542512
      easy    -1.583485
      good

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



In [123]:
tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
text_fit(X, y, tfidf_n, LogisticRegression(solver='saga', max_iter=10000))

# features: 211832
# train records: 21837
# test records: 7280
Model Accuracy: 0.7416208791208792


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.




-Top 20 positive-
         Word  Coefficient
      formula     5.168516
          gel     4.119878
          new     3.852873
      product     3.072755
          old     2.672161
     horrible     2.541212
 disappointed     2.426495
     terrible     2.365099
        money     2.107459
      changed     2.095426
       change     1.985430
        years     1.904687
         used     1.902989
  new formula     1.759747
  old formula     1.741685
        awful     1.741203
       bought     1.717925
        bring     1.697152
        worst     1.696574
       nexxus     1.677380

-Top 20 negative-
                Word  Coefficient
                 use    -1.034587
               price    -1.041470
              little    -1.068544
               color    -1.111814
              action    -1.131531
                soft    -1.244286
                best    -1.276509
               clean    -1.303570
                nice    -1.357558
                good    -1.414259
                easy 

Based on the accuracy score we can say that our previous models created having **92% accuracy** were the best since the models with Logistic regression, TF-IDF have yielded about **73% accuracy** which is lower.